In [ ]:
import torch 
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM  
import pandas as pd
from tqdm import tqdm

# Load FLAN-T5 model and tokenizer
model_name = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Move model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

print(" Model loaded on", device)

d:\Zoro_project2\zero-short-text-classification\py_3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


 Model loaded on cpu


In [4]:
df = pd.read_csv(r"D:\Zoro_project2\zero-short-text-classification\data_csv\sst2_val.csv").dropna(subset=["text"])
df["true_label"] = df["label"].map({1: "Positive", 0: "Negative"})
df.head()


,sentence,label,idx,text,true_label
0,it 's a charming and often affecting journey .,1,0,it s a charming and often affecting journey,Positive
1,unflinchingly bleak and desperate,0,1,unflinchingly bleak and desperate,Negative
2,allows us to hope that nolan is poised to emba...,1,2,allows us to hope that nolan is poised to emba...,Positive
3,"the acting , costumes , music , cinematography...",1,3,the acting costumes music cinematography and s...,Positive
4,"it 's slow -- very , very slow .",0,4,it s slow very very slow,Negative


In [5]:
import re

def load_prompt_template(path, template_number=1):
    with open(path, "r", encoding="utf-8") as f:
        content = f.read()

    # Match "Template {number}: ..." and capture until the next "Template" or EOF
    pattern = rf"Template {template_number}:(.*?)(?:Template \d+:|$)"
    match = re.search(pattern, content, re.DOTALL | re.IGNORECASE)

    if not match:
        raise ValueError(f"❌ Template {template_number} not found")

    raw_prompt = match.group(1).strip()

    # Remove leading/trailing ======= and comments
    cleaned_lines = [
        line for line in raw_prompt.splitlines()
        if line.strip() and not line.strip().startswith("#") and not line.strip().startswith("=")
    ]

    return "\n".join(cleaned_lines).strip()



In [6]:
prompt_template = load_prompt_template(r"D:\Zoro_project2\zero-short-text-classification\prompts\sst2_prompt_templates.txt", template_number=1)
print(" Extracted Prompt Template:\n", prompt_template)


 Extracted Prompt Template:
 Direct Classification Style 
Classify the sentiment of the following sentence strictly as either Positive or Negative.
Sentence: "{sentence}"
Respond with exactly one word: Positive or Negative.
Answer:


In [7]:
def generate_prompt(sentence):
    return prompt_template.replace("{sentence}", sentence)


In [8]:
def flan_predict(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_new_tokens=5)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [9]:
predictions = []

for _, row in tqdm(df.head(20).iterrows(), total=20):
    prompt = generate_prompt(row["text"])
    output = flan_predict(prompt)  #  FLAN prediction

    response = output.strip().lower()

    # Clean multiline output
    lines = [line.strip() for line in response.splitlines() if line.strip()]
    if lines:
        response = lines[-1]

    # Remove punctuation
    response = response.replace(".", "").replace(":", "").strip()

    # Normalize prediction
    if response == "positive":
        pred = "Positive"
    elif response == "negative":
        pred = "Negative"
    elif "positive" in response:
        pred = "Positive"
    elif "negative" in response:
        pred = "Negative"
    else:
        pred = "Unknown"

    print(f"\n📥 Prompt: {prompt}\n🧠 Raw Response: {output}\n✅ Cleaned Prediction: {pred}")

    
    predictions.append({
        "text": row["text"],
        "true_label": row["true_label"],
        "prediction": pred
    })




  0%|          | 0/20 [00:00<?, ?it/s]


📥 Prompt: Direct Classification Style 
Classify the sentiment of the following sentence strictly as either Positive or Negative.
Sentence: "it s a charming and often affecting journey"
Respond with exactly one word: Positive or Negative.
Answer:
🧠 Raw Response: Positive
✅ Cleaned Prediction: Positive


 10%|█         | 2/20 [01:36<13:20, 44.49s/it]


📥 Prompt: Direct Classification Style 
Classify the sentiment of the following sentence strictly as either Positive or Negative.
Sentence: "unflinchingly bleak and desperate"
Respond with exactly one word: Positive or Negative.
Answer:
🧠 Raw Response: Negative
✅ Cleaned Prediction: Negative


 15%|█▌        | 3/20 [02:03<10:21, 36.57s/it]

...


📥 Prompt: Direct Classification Style 
Classify the sentiment of the following sentence strictly as either Positive or Negative.
Sentence: "audrey tatou has a knack for picking roles that magnify her outrageous charm and in this literate french comedy she s as morningglory exuberant as she was in amlie"
Respond with exactly one word: Positive or Negative.
Answer:
🧠 Raw Response: Positive
✅ Cleaned Prediction: Positive


 95%|█████████▌| 19/20 [08:13<00:20, 20.83s/it]


📥 Prompt: Direct Classification Style 
Classify the sentiment of the following sentence strictly as either Positive or Negative.
Sentence: "the movie is just a plain old monster"
Respond with exactly one word: Positive or Negative.
Answer:
🧠 Raw Response: Negative
✅ Cleaned Prediction: Negative


100%|██████████| 20/20 [08:33<00:00, 25.69s/it]


📥 Prompt: Direct Classification Style 
Classify the sentiment of the following sentence strictly as either Positive or Negative.
Sentence: "in its best moments resembles a bad high school production of grease without benefit of song"
Respond with exactly one word: Positive or Negative.
Answer:
🧠 Raw Response: Negative
✅ Cleaned Prediction: Negative


In [ ]:
result_df = pd.DataFrame(predictions)
print("Total predictions:", len(result_df))
result_df.head()
result_df.to_csv(r"D:\Zoro_project2\zero-short-text-classification\outputs\sst2\sst2_flan_predictions_sample.csv", index=False)



Total predictions: 20


In [15]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

filtered = result_df[result_df["prediction"] != "Unknown"]

acc = accuracy_score(filtered["true_label"], filtered["prediction"])
f1 = f1_score(filtered["true_label"], filtered["prediction"], pos_label="Positive")
precision = precision_score(filtered["true_label"], filtered["prediction"], pos_label="Positive")
recall = recall_score(filtered["true_label"], filtered["prediction"], pos_label="Positive")

# Print results
print(f"\nAccuracy:  {acc:.4f}")
print(f"F1 Score:  {f1:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")

metrics = {
    "accuracy": round(acc, 4),
    "f1": round(f1, 4),
    "precision": round(precision, 4),
    "recall": round(recall, 4),
    "model": model_name
}

# Save metrics.csv
metrics_path = r"D:\Zoro_project2\zero-short-text-classification\outputs\sst2\metrics.csv"
pd.DataFrame([metrics]).to_csv(metrics_path, index=False)
print(f"Metrics saved to {metrics_path}")


# Save logs.txt

log_path = r"D:\Zoro_project2\zero-short-text-classification\outputs\sst2\logs.txt"
with open(log_path, "w", encoding="utf-8") as f:
    f.write("SST-2 Evaluation Logs\n")
    f.write("=====================\n\n")
    f.write(f"Model: {model_name}\n")
    f.write(f"Samples Evaluated: {len(result_df)}\n\n")
    f.write("Metrics:\n")
    for k, v in metrics.items():
        f.write(f"  {k}: {v}\n")
    f.write("\nSample Predictions:\n")

    # Use .get() to avoid KeyError
    for i, row in result_df.head(10).iterrows():
        raw = row.get("raw_output", "N/A")
        f.write(f"Sentence: {row['text']}\nTrue: {row['true_label']}, Pred: {row['prediction']}, Raw: {raw}\n---\n")

print(f"Logs saved to {log_path}")


Accuracy:  0.9500
F1 Score:  0.9474
Precision: 1.0000
Recall:    0.9000
Metrics saved to D:\Zoro_project2\zero-short-text-classification\outputs\sst2\metrics.csv
Logs saved to D:\Zoro_project2\zero-short-text-classification\outputs\sst2\logs.txt
